In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as patches

from keras import backend as keras
from keras.layers import *
from keras.models import Model

from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions

#from keras.preprocessing import image
#from keras.preprocessing.image import load_img
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array

from keras import backend as K

#from CNN_utils import *

plt.rcParams['figure.figsize'] = (13,13) # Make the figures a bit bigger

In [ ]:
base_model = InceptionResNetV2(include_top=True, weights='imagenet') #classes=1000

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
def get_bounds(out, percentile=95):
    # Get bounding box of 95+ percentile pixels
    a = out.flatten()
    filtered = np.array([1 if x > np.percentile(a, percentile) else 0 for x in a]).reshape(299,299)
    left, up, down, right = 299, 299, 0, 0
    for x in range(299):
        for y in range(299):
            if filtered[y,x] == 1:
                left = min(left, x)
                right = max(right, x)
                up = min(up, y)
                down = max(down, y)
    return left, up, down, right

def heatmap_for_top_pred(img2infer, model_CAM,  pred_index=None, figsizeX=12, analysed_preds=7):
    img = load_img(img2infer, target_size=(299, 299))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = model_CAM(x)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
        
    grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    heatmap = cv2.resize(heatmap.numpy(), (224,224))
    heatmap = np.uint8(255 * heatmap)

    left, up, down, right = get_bounds(heatmap, percentile=95)

    rect = patches.Rectangle((left, up), (right-left), (down-up), linewidth=1,  edgecolor='r', facecolor='none')

    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(figsizeX, figsizeX))
    axes.imshow(img, alpha=0.7)
    axes.imshow(heatmap, cmap='jet', alpha=0.3)
    left, up, down, right = get_bounds(heatmap, percentile=95)
    rect = patches.Rectangle((left, up), (right-left), (down-up), linewidth=1,  edgecolor='r', facecolor='none')
    axes.add_patch(rect)
    axes.set_title('Heat map and bounding box for prediction: '+str(decode_predictions[0][analysed_preds][1]))
    return None

In [ ]:
def get_predictions(filename):
  img = load_img(filename, target_size=(299, 299))

  plt.figure()
  plt.imshow(img)

  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  preds = base_model.predict(x)
  # decode the results into a list of tuples (class, description, probability)
  # (one such list for each sample in the batch)
  decode_preds = decode_predictions(preds, top=1)

  #print('Top 3 predictions:', decode_preds[0][0][1], decode_preds[0][1][1], decode_preds[0][2][1])
  #decode_preds[0]

  plt.title('Top prediction:'+ str(decode_preds[0][0][1]))
  return str(decode_preds[0][0][1])

In [ ]:
try:
  filename = take_photo()
  #print('Saved to {}'.format(filename))
  my_pred = get_predictions(filename)
  # Show the image which was just taken.
  #display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
#base_model.summary()
#heatmap_for_top_pred ('photo.jpg', base_model, figsizeX=12)

In [ ]:
layer_index = 'conv_7b'
model = Model(inputs=base_model.input, outputs=(base_model.get_layer(layer_index).output,base_model.layers[-1].output))

In [ ]:
figsizeX = 15

img = load_img(filename, target_size=(299, 299))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

with tf.GradientTape() as tape:
    last_conv_layer_output, preds = model(x)
    pred_index = tf.argmax(preds[0])
    class_channel = preds[:, pred_index]
    
grads = tape.gradient(class_channel, last_conv_layer_output)

pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

last_conv_layer_output = last_conv_layer_output[0]
heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
heatmap = tf.squeeze(heatmap)

heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

heatmap = cv2.resize(heatmap.numpy(), (299,299))
heatmap = np.uint8(255 * heatmap)

left, up, down, right = get_bounds(heatmap, percentile=95)

rect = patches.Rectangle((left, up), (right-left), (down-up), linewidth=1,  edgecolor='r', facecolor='none')

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(figsizeX, figsizeX))
axes.imshow(img, alpha=0.7)
axes.imshow(heatmap, cmap='jet', alpha=0.3)
left, up, down, right = get_bounds(heatmap, percentile=95)
rect = patches.Rectangle((left, up), (right-left), (down-up), linewidth=1,  edgecolor='r', facecolor='none')
axes.add_patch(rect)
axes.set_title('Heat map and bounding box for prediction: '+ my_pred)
